# Conditional recomposition ablation (large bridge model)

This standalone notebook reproduces the larger-capacity bridge experiment used to connect the analytic recomposition study to the final late-stage residual analysis.

Upload **this notebook only** to Google Colab, enable a GPU, and run all cells. No extra `.py` files or archives are required.

Model setting:

- **2 layers / width 32 / 4 heads**

Compared families:

- **ctrl3**: matched-volume control
- **highonly**: additional emphasis on upper-state structure only
- **tail**: additional lower-tail diversity only
- **tailhigh**: lower-tail data conditioned on the correct upper state

Primary question:

Does the same conditional-recomposition ordering persist at the larger capacity used for the final residual study?


In [ ]:
# ====== Basic configuration ======
RUN_PROFILE = "conditional_recomposition_large"
OUTPUT_ROOT = "results_conditional_recomposition_large"
AUTO_DOWNLOAD_ZIP = True
FORCE_CLEAN_OUTPUT_ROOT = True
SAVE_MODEL_STATE = False   # Default is False to keep the zip file compact. Set True if checkpoints are needed.

# Default hyperparameters
TARGET_EPOCH_EQUIV = 128.0   # Match the effective number of epochs across families
BATCH_SIZE = 256
LEARNING_RATE = 3e-3
WEIGHT_DECAY = 0.0
EVAL_N_PER_SUITE = 1000
BASE_SEEDS = [0, 1, 2, 3, 4]

# Fixed to the larger capacity used for the late-stage tens-residual study
CAPACITY_SPECS = [
    {"capacity_label": "large", "n_layer": 2, "n_embd": 32, "n_head": 4},
]

print({
    "RUN_PROFILE": RUN_PROFILE,
    "OUTPUT_ROOT": OUTPUT_ROOT,
    "SAVE_MODEL_STATE": SAVE_MODEL_STATE,
    "TARGET_EPOCH_EQUIV": TARGET_EPOCH_EQUIV,
    "BATCH_SIZE": BATCH_SIZE,
    "LEARNING_RATE": LEARNING_RATE,
    "WEIGHT_DECAY": WEIGHT_DECAY,
    "EVAL_N_PER_SUITE": EVAL_N_PER_SUITE,
    "BASE_SEEDS": BASE_SEEDS,
    "CAPACITY_SPECS": CAPACITY_SPECS,
})


In [ ]:

from __future__ import annotations

import json
import math
import os
import random
import shutil
import time
import zipfile
from pathlib import Path
from typing import Dict, List, Optional

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
if device == "cuda":
    print("gpu:", torch.cuda.get_device_name(0))


# ====== Data generation ======

def format_addition_doc(
    a: int,
    b: int,
    *,
    width_in: int,
    width_out: int,
) -> str:
    sa = str(a).zfill(width_in)
    sb = str(b).zfill(width_in)
    sy = str(a + b).zfill(width_out)
    return f"{sa}+{sb}={sy}"


def required_block_size(width_in: int) -> int:
    width_out = width_in + 1
    docs = [
        format_addition_doc(0, 0, width_in=width_in, width_out=width_out),
        format_addition_doc(10**width_in - 1, 10**width_in - 1, width_in=width_in, width_out=width_out),
    ]
    return max(len(doc) for doc in docs) + 1


def build_full_train_docs_2digit() -> List[str]:
    docs = []
    for a in range(100):
        for b in range(100):
            docs.append(format_addition_doc(a, b, width_in=2, width_out=3))
    return docs


def build_full_train_docs_3digit_lowrange() -> List[str]:
    docs = []
    for a in range(100):
        for b in range(100):
            docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return docs


PROBE_EXTRA_DOCS = 2000
STAGE12_EXTRA_DOCS = 2000

LOWPATTERNS_H0 = [(0, 0), (12, 23), (24, 35), (41, 18)]   # c2 = 0, c3 = 0
LOWPATTERNS_H1 = [(90, 10), (47, 53), (58, 42), (99, 1)]  # c2 = 1, c3 = 0
LOWPATTERNS_T1 = LOWPATTERNS_H0                           # isolate c3 = 1 from hundreds only

def cycle_docs_to_total(docs: List[str], total: int) -> List[str]:
    if len(docs) == 0:
        return []
    out = []
    i = 0
    while len(out) < total:
        out.append(docs[i % len(docs)])
        i += 1
    return out


def build_probe_h0_unique_docs() -> List[str]:
    docs = []
    for ha in range(10):
        for hb in range(10):
            if ha + hb == 0 or ha + hb > 9:
                continue
            for la, lb in LOWPATTERNS_H0:
                a = 100 * ha + la
                b = 100 * hb + lb
                if a < 100 and b < 100:
                    continue
                c1, c2, c3 = carry_vector_n_digits(a, b, 3)
                if c2 != 0 or c3 != 0:
                    continue
                docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return sorted(set(docs))


def build_probe_h1_unique_docs() -> List[str]:
    docs = []
    for ha in range(10):
        for hb in range(10):
            if ha + hb > 8:
                continue
            if ha == 0 and hb == 0:
                continue
            for la, lb in LOWPATTERNS_H1:
                a = 100 * ha + la
                b = 100 * hb + lb
                c1, c2, c3 = carry_vector_n_digits(a, b, 3)
                if c2 != 1 or c3 != 0:
                    continue
                docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return sorted(set(docs))


def build_probe_t1_unique_docs() -> List[str]:
    docs = []
    for ha in range(10):
        for hb in range(10):
            if ha + hb < 10:
                continue
            for la, lb in LOWPATTERNS_T1:
                a = 100 * ha + la
                b = 100 * hb + lb
                c1, c2, c3 = carry_vector_n_digits(a, b, 3)
                if c2 != 0 or c3 != 1:
                    continue
                docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return sorted(set(docs))



def build_ctrl_dup_docs(total: int = PROBE_EXTRA_DOCS, seed: int = 777) -> List[str]:
    rnd = random.Random(seed)
    base = build_full_train_docs_3digit_lowrange()
    return [base[rnd.randrange(len(base))] for _ in range(total)]


def build_ctrl2_docs(total: int = STAGE12_EXTRA_DOCS, seed: int = 1707) -> List[str]:
    rnd = random.Random(seed)
    base = build_full_train_docs_2digit() + build_full_train_docs_3digit_lowrange()
    return [base[rnd.randrange(len(base))] for _ in range(total)]


def build_probe_t1c2_1_unique_docs() -> List[str]:
    docs = []
    for ha in range(10):
        for hb in range(10):
            if ha + hb < 9:
                continue
            for la, lb in LOWPATTERNS_H1:
                a = 100 * ha + la
                b = 100 * hb + lb
                if a < 100 and b < 100:
                    continue
                c1, c2, c3 = carry_vector_n_digits(a, b, 3)
                if c2 != 1 or c3 != 1:
                    continue
                docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return sorted(set(docs))


A_ORDER = [((137 * i) + 53) % 1000 for i in range(1000)]
B_ORDER = [((353 * i) + 97) % 1000 for i in range(1000)]


def collect_bridge_docs(
    *,
    incoming_carry_to_hundreds: Optional[int] = None,
    carryout_from_hundreds: Optional[int] = None,
    bucket_mode: str = "low2_c1",
) -> List[str]:
    docs = []
    seen = set()
    for a in A_ORDER:
        for b in B_ORDER:
            if a < 100 and b < 100:
                continue
            c1, c2, c3 = carry_vector_n_digits(a, b, 3)
            if incoming_carry_to_hundreds is not None and c2 != incoming_carry_to_hundreds:
                continue
            if carryout_from_hundreds is not None and c3 != carryout_from_hundreds:
                continue

            low2 = (a + b) % 100
            if bucket_mode == "low2_c1":
                bucket = (low2, c1)
            elif bucket_mode == "low2_c2":
                bucket = (low2, c2)
            else:
                raise ValueError(bucket_mode)

            if bucket in seen:
                continue
            seen.add(bucket)
            docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return docs


def build_bridge_unique_docs() -> List[str]:
    h0 = collect_bridge_docs(
        incoming_carry_to_hundreds=0,
        carryout_from_hundreds=0,
        bucket_mode="low2_c1",
    )
    h1 = collect_bridge_docs(
        incoming_carry_to_hundreds=1,
        carryout_from_hundreds=0,
        bucket_mode="low2_c1",
    )
    t1 = collect_bridge_docs(
        carryout_from_hundreds=1,
        bucket_mode="low2_c2",
    )
    return sorted(set(h0 + h1 + t1))


def collect_tail_docs(
    *,
    require_corner: bool = False,
) -> List[str]:
    docs = []
    seen = set()
    for a in A_ORDER:
        for b in B_ORDER:
            if a < 100 and b < 100:
                continue
            c1, c2, c3 = carry_vector_n_digits(a, b, 3)
            if require_corner and not (c2 == 1 or c3 == 1):
                continue
            total = a + b
            tens_out = (total // 10) % 10
            ones_out = total % 10
            bucket = (c1, c2, c3, tens_out, ones_out)
            if bucket in seen:
                continue
            seen.add(bucket)
            docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return docs


def build_tail_unique_docs() -> List[str]:
    return sorted(set(collect_tail_docs(require_corner=False)))


def build_tailcorner_unique_docs() -> List[str]:
    return sorted(set(collect_tail_docs(require_corner=True)))

def collect_tailhigh_docs(
    *,
    require_corner: bool = False,
) -> List[str]:
    docs = []
    seen = set()
    for a in A_ORDER:
        for b in B_ORDER:
            if a < 100 and b < 100:
                continue
            c1, c2, c3 = carry_vector_n_digits(a, b, 3)
            if require_corner and not (c2 == 1 or c3 == 1):
                continue
            total = a + b
            thousands_out = (total // 1000) % 10
            hundreds_out = (total // 100) % 10
            tens_out = (total // 10) % 10
            ones_out = total % 10
            bucket = (thousands_out, hundreds_out, c1, c2, c3, tens_out, ones_out)
            if bucket in seen:
                continue
            seen.add(bucket)
            docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return docs


def build_tailhigh_unique_docs() -> List[str]:
    return sorted(set(collect_tailhigh_docs(require_corner=False)))


def build_tailhighcorner_unique_docs() -> List[str]:
    return sorted(set(collect_tailhigh_docs(require_corner=True)))


def collect_highonly_docs(
    *,
    require_corner: bool = False,
) -> List[str]:
    docs = []
    seen = set()
    for a in A_ORDER:
        for b in B_ORDER:
            if a < 100 and b < 100:
                continue
            c1, c2, c3 = carry_vector_n_digits(a, b, 3)
            if require_corner and not (c2 == 1 or c3 == 1):
                continue
            total = a + b
            thousands_out = (total // 1000) % 10
            hundreds_out = (total // 100) % 10
            bucket = (thousands_out, hundreds_out, c1, c2, c3)
            if bucket in seen:
                continue
            seen.add(bucket)
            docs.append(format_addition_doc(a, b, width_in=3, width_out=4))
    return docs


def build_highonly_unique_docs() -> List[str]:
    return sorted(set(collect_highonly_docs(require_corner=False)))


def build_probe_pack(kind: str, total: int = PROBE_EXTRA_DOCS) -> List[str]:
    if kind == "ctrl_dup":
        return build_ctrl_dup_docs(total=total)
    if kind == "probe_h0":
        return cycle_docs_to_total(build_probe_h0_unique_docs(), total)
    if kind == "probe_h1":
        return cycle_docs_to_total(build_probe_h1_unique_docs(), total)
    if kind == "probe_t1":
        return cycle_docs_to_total(build_probe_t1_unique_docs(), total)
    if kind == "probe_all":
        h0 = cycle_docs_to_total(build_probe_h0_unique_docs(), total // 3)
        h1 = cycle_docs_to_total(build_probe_h1_unique_docs(), total // 3)
        t1 = cycle_docs_to_total(build_probe_t1_unique_docs(), total - 2 * (total // 3))
        return h0 + h1 + t1
    if kind == "ctrl2":
        return build_ctrl2_docs(total=total)
    if kind == "probe_t1c2_1":
        return cycle_docs_to_total(build_probe_t1c2_1_unique_docs(), total)
    if kind == "bridge":
        return cycle_docs_to_total(build_bridge_unique_docs(), total)
    if kind == "tail":
        return cycle_docs_to_total(build_tail_unique_docs(), total)
    if kind == "tailcorner":
        return cycle_docs_to_total(build_tailcorner_unique_docs(), total)
    if kind == "tailhigh":
        return cycle_docs_to_total(build_tailhigh_unique_docs(), total)
    if kind == "highonly":
        return cycle_docs_to_total(build_highonly_unique_docs(), total)
    if kind == "tailhighcorner":
        return cycle_docs_to_total(build_tailhighcorner_unique_docs(), total)
    return []



def sample_pairs_2digit(n: int, seed: int) -> List[tuple[int, int]]:
    rnd = random.Random(seed)
    return [(rnd.randrange(100), rnd.randrange(100)) for _ in range(n)]


def carry_vector_n_digits(a: int, b: int, n_digits: int) -> tuple[int, ...]:
    c = 0
    vec = []
    for i in range(n_digits):
        da = (a // (10 ** i)) % 10
        db = (b // (10 ** i)) % 10
        total = da + db + c
        c = 1 if total >= 10 else 0
        vec.append(c)
    return tuple(vec)


def sample_pairs_true_3digit_conditioned(
    n: int,
    seed: int,
    *,
    incoming_carry_to_hundreds: Optional[int] = None,
    carryout_from_hundreds: Optional[int] = None,
) -> List[tuple[int, int]]:
    rnd = random.Random(seed)
    rows = []
    while len(rows) < n:
        a = rnd.randrange(1000)
        b = rnd.randrange(1000)
        if a < 100 and b < 100:
            continue
        c1, c2, c3 = carry_vector_n_digits(a, b, 3)
        if incoming_carry_to_hundreds is not None and c2 != incoming_carry_to_hundreds:
            continue
        if carryout_from_hundreds is not None and c3 != carryout_from_hundreds:
            continue
        rows.append((a, b))
    return rows


def build_eval_rows_from_pairs(
    pairs: List[tuple[int, int]],
    *,
    width_in: int,
    width_out: int,
    suite: str,
) -> List[dict]:
    rows = []
    for a, b in pairs:
        doc = format_addition_doc(a, b, width_in=width_in, width_out=width_out)
        prefix, target = doc.split("=")
        rows.append({
            "suite": suite,
            "a": a,
            "b": b,
            "doc": doc,
            "prefix": prefix + "=",
            "target": target,
            "y": a + b,
            "width_in": width_in,
            "width_out": width_out,
            "style": "zero_pad",
        })
    return rows


def build_abs_position_role_aliasing_table():
    def role_table(width_in: int):
        roles = [("BOS", 0)]
        for i in range(width_in):
            roles.append((f"a_{width_in - 1 - i}", len(roles)))
        roles.append(("+", len(roles)))
        for i in range(width_in):
            roles.append((f"b_{width_in - 1 - i}", len(roles)))
        roles.append(("=", len(roles)))
        width_out = width_in + 1
        for i in range(width_out):
            roles.append((f"y_{width_out - 1 - i}", len(roles)))
        return pd.DataFrame(roles, columns=["role", "abs_pos"])

    train2 = role_table(2)
    low3 = role_table(3)
    rows = []
    max_pos = max(train2["abs_pos"].max(), low3["abs_pos"].max())
    for p in range(max_pos + 1):
        tr = train2.loc[train2["abs_pos"] == p, "role"]
        te = low3.loc[low3["abs_pos"] == p, "role"]
        tr_role = tr.iloc[0] if len(tr) else "(unused/masked)"
        te_role = te.iloc[0] if len(te) else "(unused)"
        rows.append({
            "abs_pos": p,
            "train_full2_role": tr_role,
            "test_lowrange3_role": te_role,
            "full2_supervision": "yes" if tr_role != "(unused/masked)" else "no",
            "role_shift_type": "same" if tr_role == te_role else ("unseen_in_full2" if tr_role == "(unused/masked)" else "changed"),
        })
    return pd.DataFrame(rows)


def build_abs_position_support_table():
    rows = []
    block_size = max(required_block_size(2), required_block_size(3))
    for p in range(block_size):
        rows.append({
            "abs_pos": p,
            "supervised_in_full2": int(p <= 9),
            "supervised_in_mixedlayout_abspos": int(p <= 12),
            "note": (
                "used_by_2digit_train" if p <= 9 else
                ("unused_in_full2_but_used_by_3digit_layout" if p <= 12 else "outside_position_block")
            ),
        })
    return pd.DataFrame(rows)


def build_coupled_position_scheme_table():
    rows = [
        {"pos_id": 0, "semantic": "BOS"},
        {"pos_id": 1, "semantic": "+"},
        {"pos_id": 2, "semantic": "="},
        {"pos_id": 3, "semantic": "digit_sig_0 (ones)"},
        {"pos_id": 4, "semantic": "digit_sig_1 (tens)"},
        {"pos_id": 5, "semantic": "digit_sig_2 (hundreds)"},
        {"pos_id": 6, "semantic": "digit_sig_3 (thousands)"},
    ]
    return pd.DataFrame(rows)


ABS_POSITION_ROLE_ALIASING = build_abs_position_role_aliasing_table()
ABS_POSITION_SUPPORT = build_abs_position_support_table()
COUPLED_POSITION_SCHEME = build_coupled_position_scheme_table()

print(ABS_POSITION_ROLE_ALIASING)
print(COUPLED_POSITION_SCHEME)


# ====== Position IDs ======

def coupled_pos_ids_from_text(text: str) -> List[int]:
    # ids are computed on [BOS] + text
    # output digit ids use the FULL expected output width (= input width + 1),
    # so partial generation remains stable.
    if "+" not in text:
        raise ValueError(f"invalid text (missing +): {text}")
    if "=" in text:
        left, right = text.split("=", 1)
    else:
        left, right = text, ""
    a, b = left.split("+", 1)
    width_in = len(a)
    width_out = width_in + 1

    ids = [0]  # BOS
    for i, _ch in enumerate(a):
        sig = len(a) - 1 - i
        ids.append(3 + sig)
    ids.append(1)  # plus
    for i, _ch in enumerate(b):
        sig = len(b) - 1 - i
        ids.append(3 + sig)
    if "=" in text:
        ids.append(2)  # equals
        for i, _ch in enumerate(right):
            sig = width_out - 1 - i
            ids.append(3 + sig)
    return ids


# ====== Model ======

class RMSNorm(nn.Module):
    def __init__(self, dim: int, eps: float = 1e-5):
        super().__init__()
        self.eps = eps

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return x * torch.rsqrt(x.pow(2).mean(dim=-1, keepdim=True) + self.eps)


class Block(nn.Module):
    def __init__(self, n_embd: int, n_head: int, block_size: int):
        super().__init__()
        assert n_embd % n_head == 0
        self.n_head = n_head
        self.head_dim = n_embd // n_head
        self.block_size = block_size

        self.attn_norm = RMSNorm(n_embd)
        self.wq = nn.Linear(n_embd, n_embd, bias=False)
        self.wk = nn.Linear(n_embd, n_embd, bias=False)
        self.wv = nn.Linear(n_embd, n_embd, bias=False)
        self.wo = nn.Linear(n_embd, n_embd, bias=False)

        self.mlp_norm = RMSNorm(n_embd)
        self.fc1 = nn.Linear(n_embd, 4 * n_embd, bias=False)
        self.fc2 = nn.Linear(4 * n_embd, n_embd, bias=False)

        self.register_buffer("mask", torch.tril(torch.ones(block_size, block_size, dtype=torch.bool)))

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        B, T, C = x.shape

        x_res = x
        xn = self.attn_norm(x)
        q = self.wq(xn).view(B, T, self.n_head, self.head_dim).transpose(1, 2)
        k = self.wk(xn).view(B, T, self.n_head, self.head_dim).transpose(1, 2)
        v = self.wv(xn).view(B, T, self.n_head, self.head_dim).transpose(1, 2)

        att = (q @ k.transpose(-2, -1)) / math.sqrt(self.head_dim)
        att = att.masked_fill(~self.mask[:T, :T], float("-inf"))
        att = F.softmax(att, dim=-1)

        y = (att @ v).transpose(1, 2).contiguous().view(B, T, C)
        y = self.wo(y)
        x = x_res + y

        x_res = x
        xn = self.mlp_norm(x)
        y = self.fc2(F.relu(self.fc1(xn)))
        x = x_res + y
        return x


class TinyAbsPos(nn.Module):
    def __init__(self, vocab_size: int, block_size: int, n_layer: int, n_embd: int, n_head: int):
        super().__init__()
        self.block_size = block_size
        self.wte = nn.Embedding(vocab_size, n_embd)
        self.wpe = nn.Embedding(block_size, n_embd)
        self.in_norm = RMSNorm(n_embd)
        self.blocks = nn.ModuleList([Block(n_embd, n_head, block_size) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, idx: torch.Tensor) -> torch.Tensor:
        B, T = idx.shape
        pos = torch.arange(T, device=idx.device)[None, :]
        x = self.wte(idx) + self.wpe(pos)
        x = self.in_norm(x)
        for block in self.blocks:
            x = block(x)
        return self.lm_head(x)


class TinyNoPE(nn.Module):
    def __init__(self, vocab_size: int, block_size: int, n_layer: int, n_embd: int, n_head: int):
        super().__init__()
        self.block_size = block_size
        self.wte = nn.Embedding(vocab_size, n_embd)
        self.in_norm = RMSNorm(n_embd)
        self.blocks = nn.ModuleList([Block(n_embd, n_head, block_size) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, idx: torch.Tensor) -> torch.Tensor:
        x = self.wte(idx)
        x = self.in_norm(x)
        for block in self.blocks:
            x = block(x)
        return self.lm_head(x)


class TinyCoupledSigPos(nn.Module):
    def __init__(self, vocab_size: int, pos_vocab_size: int, block_size: int, n_layer: int, n_embd: int, n_head: int):
        super().__init__()
        self.block_size = block_size
        self.wte = nn.Embedding(vocab_size, n_embd)
        self.pte = nn.Embedding(pos_vocab_size, n_embd)
        self.in_norm = RMSNorm(n_embd)
        self.blocks = nn.ModuleList([Block(n_embd, n_head, block_size) for _ in range(n_layer)])
        self.lm_head = nn.Linear(n_embd, vocab_size, bias=False)

    def forward(self, idx: torch.Tensor, pos_ids: torch.Tensor) -> torch.Tensor:
        x = self.wte(idx) + self.pte(pos_ids)
        x = self.in_norm(x)
        for block in self.blocks:
            x = block(x)
        return self.lm_head(x)


POS_VOCAB_SIZE_COUPLED = int(COUPLED_POSITION_SCHEME["pos_id"].max()) + 1


# ====== Tokenization / tensorization ======

def build_vocab(docs: List[str]):
    uchars = sorted(set("".join(docs)))
    stoi = {ch: i for i, ch in enumerate(uchars)}
    itos = {i: ch for ch, i in stoi.items()}
    BOS = len(uchars)
    vocab_size = len(uchars) + 1
    return uchars, stoi, itos, BOS, vocab_size


def docs_to_tensors(docs: List[str], stoi: Dict[str, int], BOS: int, block_size: int, model_kind: str):
    xs, ys, ps = [], [], []
    for doc in docs:
        tokens = [BOS] + [stoi[ch] for ch in doc] + [BOS]
        x = tokens[:-1]
        y = tokens[1:]

        if model_kind == "coupled_sig":
            p = coupled_pos_ids_from_text(doc)
            assert len(p) == len(x), (doc, len(p), len(x))
        else:
            p = [0] * len(x)

        if len(x) > block_size:
            x = x[:block_size]
            y = y[:block_size]
            p = p[:block_size]

        pad = block_size - len(x)
        x = x + [BOS] * pad
        y = y + [-100] * pad
        p = p + [0] * pad

        xs.append(x)
        ys.append(y)
        ps.append(p)

    return (
        torch.tensor(xs, dtype=torch.long),
        torch.tensor(ys, dtype=torch.long),
        torch.tensor(ps, dtype=torch.long),
    )


def set_global_seed(seed: int):
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)


# ====== train / eval ======

def build_model(model_kind: str, vocab_size: int, block_size: int, n_layer: int, n_embd: int, n_head: int):
    if model_kind == "abspos":
        return TinyAbsPos(vocab_size, block_size, n_layer, n_embd, n_head)
    if model_kind == "nope":
        return TinyNoPE(vocab_size, block_size, n_layer, n_embd, n_head)
    if model_kind == "coupled_sig":
        return TinyCoupledSigPos(vocab_size, POS_VOCAB_SIZE_COUPLED, block_size, n_layer, n_embd, n_head)
    raise ValueError(model_kind)


def model_forward(model_kind: str, model: nn.Module, idx: torch.Tensor, pos_ids: Optional[torch.Tensor] = None) -> torch.Tensor:
    if model_kind == "coupled_sig":
        assert pos_ids is not None
        return model(idx, pos_ids)
    return model(idx)


def sample_batch(xs: torch.Tensor, ys: torch.Tensor, ps: torch.Tensor, batch_size: int, device: str):
    idx = torch.randint(low=0, high=len(xs), size=(batch_size,))
    return xs[idx].to(device), ys[idx].to(device), ps[idx].to(device)


def current_pos_tensor_for_text(text: str, model_kind: str, device: str):
    if model_kind != "coupled_sig":
        return None
    pos_ids = coupled_pos_ids_from_text(text)
    return torch.tensor(pos_ids, dtype=torch.long, device=device).unsqueeze(0)


def greedy_generate_free(model_kind, model, prefix: str, stoi, itos, BOS: int, block_size: int, device: str, max_new_tokens: int):
    model.eval()
    ids = [BOS] + [stoi[ch] for ch in prefix]
    pred = []
    with torch.no_grad():
        for _ in range(max_new_tokens):
            x = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)
            pos_ids = current_pos_tensor_for_text(prefix + "".join(pred), model_kind, device)
            logits = model_forward(model_kind, model, x, pos_ids)[:, -1, :]
            token_id = int(torch.argmax(logits, dim=-1).item())
            if token_id == BOS:
                break
            pred.append(itos[token_id])
            ids.append(token_id)
            if len(ids) >= block_size:
                break
    return "".join(pred)


def greedy_generate_digits_fixedlen(model_kind, model, prefix: str, stoi, itos, BOS: int, block_size: int, device: str, exact_len: int):
    model.eval()
    ids = [BOS] + [stoi[ch] for ch in prefix]
    digit_token_ids = [stoi[str(i)] for i in range(10)]
    pred = []
    with torch.no_grad():
        for _ in range(exact_len):
            x = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)
            pos_ids = current_pos_tensor_for_text(prefix + "".join(pred), model_kind, device)
            logits = model_forward(model_kind, model, x, pos_ids)[:, -1, :].clone()
            mask = torch.full_like(logits, float("-inf"))
            mask[:, digit_token_ids] = 0.0
            logits = logits + mask
            token_id = int(torch.argmax(logits, dim=-1).item())
            pred.append(itos[token_id])
            ids.append(token_id)
            if len(ids) >= block_size:
                break
    return "".join(pred)


def teacher_forced_detail(model_kind, model, prefix: str, target: str, stoi, itos, BOS: int, block_size: int, device: str):
    model.eval()
    ids = [BOS] + [stoi[ch] for ch in prefix]
    current_text = prefix
    correct = []
    pred_chars = []
    with torch.no_grad():
        for ch in target:
            x = torch.tensor(ids, dtype=torch.long, device=device).unsqueeze(0)
            pos_ids = current_pos_tensor_for_text(current_text, model_kind, device)
            logits = model_forward(model_kind, model, x, pos_ids)[:, -1, :]
            pred_id = int(torch.argmax(logits, dim=-1).item())
            pred_char = None if pred_id == BOS else str(itos[pred_id])
            target_id = stoi[ch]
            correct.append(int(pred_id == target_id))
            pred_chars.append(pred_char)
            ids.append(target_id)
            current_text += ch
            if len(ids) >= block_size:
                break
    return correct, pred_chars


# ====== Experiment definitions ======





def build_design_manifest():
    family_kinds = [
        {
            "family_stub": "mixedlayout_coupledsig_probeall_bridge_t1c2_tailcorner_ctrl3",
            "train_family": "mixedlayout",
            "model_kind": "coupled_sig",
            "extra_data_kind": "probe_all_bridge_t1c2_tailcorner_ctrl3",
            "extra_docs_count": PROBE_EXTRA_DOCS + 4 * STAGE12_EXTRA_DOCS,
            "uses_targeted_counterexamples": 0,
            "target_pack_label": "matched_volume_control_on_top_of_tailcorner",
        },
        {
            "family_stub": "mixedlayout_coupledsig_probeall_bridge_t1c2_tailcorner_tail",
            "train_family": "mixedlayout",
            "model_kind": "coupled_sig",
            "extra_data_kind": "probe_all_bridge_t1c2_tailcorner_tail",
            "extra_docs_count": PROBE_EXTRA_DOCS + 4 * STAGE12_EXTRA_DOCS,
            "uses_targeted_counterexamples": 1,
            "target_pack_label": "tail_only_repair",
        },
        {
            "family_stub": "mixedlayout_coupledsig_probeall_bridge_t1c2_tailcorner_highonly",
            "train_family": "mixedlayout",
            "model_kind": "coupled_sig",
            "extra_data_kind": "probe_all_bridge_t1c2_tailcorner_highonly",
            "extra_docs_count": PROBE_EXTRA_DOCS + 4 * STAGE12_EXTRA_DOCS,
            "uses_targeted_counterexamples": 1,
            "target_pack_label": "high_only_repair",
        },
        {
            "family_stub": "mixedlayout_coupledsig_probeall_bridge_t1c2_tailcorner_tailhigh",
            "train_family": "mixedlayout",
            "model_kind": "coupled_sig",
            "extra_data_kind": "probe_all_bridge_t1c2_tailcorner_tailhigh",
            "extra_docs_count": PROBE_EXTRA_DOCS + 4 * STAGE12_EXTRA_DOCS,
            "uses_targeted_counterexamples": 1,
            "target_pack_label": "high_conditioned_tail_repair",
        },
    ]
    rows = []
    for cap in CAPACITY_SPECS:
        for spec in family_kinds:
            rows.append({
                "family": f"{spec['family_stub']}_{cap['capacity_label']}",
                "family_stub": spec["family_stub"],
                "capacity_label": cap["capacity_label"],
                "train_family": spec["train_family"],
                "model_kind": spec["model_kind"],
                "extra_data_kind": spec["extra_data_kind"],
                "extra_docs_count": spec["extra_docs_count"],
                "target_pack_label": spec["target_pack_label"],
                "n_layer": cap["n_layer"],
                "n_embd": cap["n_embd"],
                "n_head": cap["n_head"],
                "uses_explicit_absolute_positions": 0,
                "uses_no_position_embedding": 0,
                "uses_coupled_significance_positions": 1,
                "sees_3digit_layout_in_train": 1,
                "uses_targeted_counterexamples": spec["uses_targeted_counterexamples"],
            })
    return pd.DataFrame(rows)

DESIGN_MANIFEST = build_design_manifest()
print(DESIGN_MANIFEST)





def build_train_docs(train_family: str, extra_data_kind: str = "probe_all_bridge_t1c2_tailcorner_ctrl3") -> List[str]:
    if train_family != "mixedlayout":
        raise ValueError(train_family)

    base_docs = build_full_train_docs_2digit() + build_full_train_docs_3digit_lowrange()
    docs = list(base_docs) + build_probe_pack("probe_all", total=PROBE_EXTRA_DOCS)
    docs = docs + build_probe_pack("bridge", total=STAGE12_EXTRA_DOCS)
    docs = docs + build_probe_pack("probe_t1c2_1", total=STAGE12_EXTRA_DOCS)
    docs = docs + build_probe_pack("tailcorner", total=STAGE12_EXTRA_DOCS)

    if extra_data_kind == "probe_all_bridge_t1c2_tailcorner_ctrl3":
        return docs + build_probe_pack("ctrl2", total=STAGE12_EXTRA_DOCS)
    if extra_data_kind == "probe_all_bridge_t1c2_tailcorner_tail":
        return docs + build_probe_pack("tail", total=STAGE12_EXTRA_DOCS)
    if extra_data_kind == "probe_all_bridge_t1c2_tailcorner_highonly":
        return docs + build_probe_pack("highonly", total=STAGE12_EXTRA_DOCS)
    if extra_data_kind == "probe_all_bridge_t1c2_tailcorner_tailhigh":
        return docs + build_probe_pack("tailhigh", total=STAGE12_EXTRA_DOCS)
    raise ValueError(extra_data_kind)



def build_experiments():
    experiments = []
    block_size = max(required_block_size(2), required_block_size(3))  # 13

    for row in DESIGN_MANIFEST.to_dict(orient="records"):
        train_docs = build_train_docs(row["train_family"], extra_data_kind=row["extra_data_kind"])
        epoch_target = float(TARGET_EPOCH_EQUIV)
        num_steps = int(math.ceil(epoch_target * len(train_docs) / max(1, BATCH_SIZE)))
        if num_steps % 250 != 0:
            num_steps = 250 * int(math.ceil(num_steps / 250))

        for seed in BASE_SEEDS:
            eval_rows = []
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_2digit(EVAL_N_PER_SUITE, seed=1000 + seed),
                width_in=2, width_out=3, suite="in_support_2digit",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_2digit(EVAL_N_PER_SUITE, seed=2000 + seed),
                width_in=3, width_out=4, suite="layout_shift_only_lowrange3",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE, seed=3000 + seed,
                    incoming_carry_to_hundreds=0,
                    carryout_from_hundreds=0,
                ),
                width_in=3, width_out=4, suite="true3_hundreds_no_incarry",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE, seed=4000 + seed,
                    incoming_carry_to_hundreds=1,
                    carryout_from_hundreds=0,
                ),
                width_in=3, width_out=4, suite="true3_hundreds_with_incarry",
            ))
            eval_rows.extend(build_eval_rows_from_pairs(
                sample_pairs_true_3digit_conditioned(
                    EVAL_N_PER_SUITE, seed=5000 + seed,
                    carryout_from_hundreds=1,
                ),
                width_in=3, width_out=4, suite="true3_thousands_carry",
            ))
            experiments.append({
                "name": f"{row['family']}_seed{seed}",
                "family": row["family"],
                "train_family": row["train_family"],
                "model_kind": row["model_kind"],
                "split": "conditional_recomposition_large_pack",
                "style": "zero_pad",
                "seed": seed,
                "block_size": block_size,
                "num_steps": num_steps,
                "batch_size": BATCH_SIZE,
                "learning_rate": LEARNING_RATE,
                "weight_decay": WEIGHT_DECAY,
                "n_layer": int(row["n_layer"]),
                "n_embd": int(row["n_embd"]),
                "n_head": int(row["n_head"]),
                "train_docs": train_docs,
                "eval_rows": eval_rows,
                "n_train_docs": len(train_docs),
                "n_eval": len(eval_rows),
                "extra_data_kind": row["extra_data_kind"],
                "extra_docs_count": int(row["extra_docs_count"]),
                "capacity_label": row["capacity_label"],
                "family_stub": row["family_stub"],
            })
    return experiments


EXPERIMENTS = build_experiments()
print(pd.DataFrame([{
    "name": e["name"],
    "family": e["family"],
    "train_family": e["train_family"],
    "model_kind": e["model_kind"],
    "seed": e["seed"],
    "block_size": e["block_size"],
    "num_steps": e["num_steps"],
    "n_train_docs": e["n_train_docs"],
    "n_eval": e["n_eval"],
} for e in EXPERIMENTS]))


# ====== Run ======

output_root = Path(OUTPUT_ROOT)

if FORCE_CLEAN_OUTPUT_ROOT and output_root.exists():
    shutil.rmtree(output_root)
output_root.mkdir(parents=True, exist_ok=True)

ABS_POSITION_ROLE_ALIASING.to_csv(output_root / "absolute_position_role_aliasing.csv", index=False)
ABS_POSITION_SUPPORT.to_csv(output_root / "absolute_position_support.csv", index=False)
COUPLED_POSITION_SCHEME.to_csv(output_root / "coupled_position_scheme.csv", index=False)
DESIGN_MANIFEST.to_csv(output_root / "design_manifest.csv", index=False)


probe_manifest_rows = [
    {"probe_kind": "probe_h0", "unique_docs": len(build_probe_h0_unique_docs()), "added_docs_after_stretch": PROBE_EXTRA_DOCS},
    {"probe_kind": "probe_h1", "unique_docs": len(build_probe_h1_unique_docs()), "added_docs_after_stretch": PROBE_EXTRA_DOCS},
    {"probe_kind": "probe_t1", "unique_docs": len(build_probe_t1_unique_docs()), "added_docs_after_stretch": PROBE_EXTRA_DOCS},
    {"probe_kind": "probe_t1c2_1", "unique_docs": len(build_probe_t1c2_1_unique_docs()), "added_docs_after_stretch": STAGE12_EXTRA_DOCS},
    {"probe_kind": "bridge", "unique_docs": len(build_bridge_unique_docs()), "added_docs_after_stretch": STAGE12_EXTRA_DOCS},
    {"probe_kind": "tail", "unique_docs": len(build_tail_unique_docs()), "added_docs_after_stretch": STAGE12_EXTRA_DOCS},
    {"probe_kind": "highonly", "unique_docs": len(build_highonly_unique_docs()), "added_docs_after_stretch": STAGE12_EXTRA_DOCS},
    {"probe_kind": "tailhigh", "unique_docs": len(build_tailhigh_unique_docs()), "added_docs_after_stretch": STAGE12_EXTRA_DOCS},
]
pd.DataFrame(probe_manifest_rows).to_csv(output_root / "probe_design_manifest.csv", index=False)
(output_root / "probe_h0_preview.txt").write_text("\n".join(build_probe_h0_unique_docs()[:80]), encoding="utf-8")
(output_root / "probe_h1_preview.txt").write_text("\n".join(build_probe_h1_unique_docs()[:80]), encoding="utf-8")
(output_root / "probe_t1_preview.txt").write_text("\n".join(build_probe_t1_unique_docs()[:80]), encoding="utf-8")
(output_root / "probe_t1c2_1_preview.txt").write_text("\n".join(build_probe_t1c2_1_unique_docs()[:80]), encoding="utf-8")
(output_root / "bridge_preview.txt").write_text("\n".join(build_bridge_unique_docs()[:80]), encoding="utf-8")
(output_root / "tail_preview.txt").write_text("\n".join(build_tail_unique_docs()[:80]), encoding="utf-8")
(output_root / "highonly_preview.txt").write_text("\n".join(build_highonly_unique_docs()[:80]), encoding="utf-8")
(output_root / "tailhigh_preview.txt").write_text("\n".join(build_tailhigh_unique_docs()[:80]), encoding="utf-8")

manifest_rows = []
combined_rows = []
suite_rows = []
per_position_rows = []
teacher_token_rows = []
carry_probe_rows = []
recomposition_rows = []

for exp in EXPERIMENTS:
    name = exp["name"]
    workdir = output_root / name
    workdir.mkdir(parents=True, exist_ok=True)

    train_docs = exp["train_docs"]
    eval_rows = exp["eval_rows"]
    seed = int(exp["seed"])
    block_size = int(exp["block_size"])
    num_steps = int(exp["num_steps"])
    model_kind = str(exp["model_kind"])

    set_global_seed(seed)

    uchars, stoi, itos, BOS, vocab_size = build_vocab(train_docs)
    xs, ys, ps = docs_to_tensors(train_docs, stoi, BOS, block_size=block_size, model_kind=model_kind)

    model = build_model(
        model_kind=model_kind,
        vocab_size=vocab_size,
        block_size=block_size,
        n_layer=int(exp["n_layer"]),
        n_embd=int(exp["n_embd"]),
        n_head=int(exp["n_head"]),
    ).to(device)

    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=float(exp["learning_rate"]),
        weight_decay=float(exp["weight_decay"]),
    )

    config = {
        "name": name,
        "family": exp["family"],
        "train_family": exp["train_family"],
        "model_kind": exp["model_kind"],
        "split": exp["split"],
        "style": exp["style"],
        "seed": seed,
        "block_size": block_size,
        "num_steps": num_steps,
        "batch_size": int(exp["batch_size"]),
        "learning_rate": float(exp["learning_rate"]),
        "weight_decay": float(exp["weight_decay"]),
        "n_layer": int(exp["n_layer"]),
        "n_embd": int(exp["n_embd"]),
        "n_head": int(exp["n_head"]),
        "n_train_docs": len(train_docs),
        "n_eval": len(eval_rows),
        "device": device,
        "vocab_size": vocab_size,
        "epoch_equiv": float(num_steps * exp["batch_size"] / max(1, len(train_docs))),
        "extra_data_kind": exp["extra_data_kind"],
        "extra_docs_count": int(exp["extra_docs_count"]),
        "capacity_label": exp.get("capacity_label", ""),
        "family_stub": exp.get("family_stub", ""),
    }
    manifest_rows.append(config.copy())

    (workdir / "config.json").write_text(json.dumps(config, ensure_ascii=False, indent=2), encoding="utf-8")
    (workdir / "vocab.json").write_text(json.dumps({"uchars": uchars, "stoi": stoi, "itos": itos, "BOS": BOS}, ensure_ascii=False, indent=2), encoding="utf-8")
    (workdir / "input_train_preview.txt").write_text("\n".join(train_docs[:50]), encoding="utf-8")

    start = time.time()
    train_log = []
    model.train()
    for step in range(1, num_steps + 1):
        xb, yb, pb = sample_batch(xs, ys, ps, int(exp["batch_size"]), device)
        logits = model_forward(model_kind, model, xb, pb)
        loss = F.cross_entropy(logits.view(-1, vocab_size), yb.view(-1), ignore_index=-100)
        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        optimizer.step()

        if step == 1 or step % 250 == 0 or step == num_steps:
            train_log.append({"step": step, "loss": float(loss.item())})

    runtime_sec = time.time() - start
    pd.DataFrame(train_log).to_csv(workdir / "train_log.csv", index=False)

    if SAVE_MODEL_STATE:
        torch.save(model.state_dict(), workdir / "model_state.pt")

    # evaluation
    pred_records = []
    for row in eval_rows:
        pred_free = greedy_generate_free(
            model_kind, model, row["prefix"], stoi, itos, BOS, block_size, device, max_new_tokens=len(row["target"])
        )
        pred_constrained = greedy_generate_digits_fixedlen(
            model_kind, model, row["prefix"], stoi, itos, BOS, block_size, device, exact_len=len(row["target"])
        )
        teacher_correct, teacher_pred_chars = teacher_forced_detail(
            model_kind, model, row["prefix"], row["target"], stoi, itos, BOS, block_size, device
        )

        rec = dict(row)
        rec.update({
            "pred_free": pred_free,
            "pred_constrained": pred_constrained,
            "exact_free": int(pred_free == row["target"]),
            "exact_constrained": int(pred_constrained == row["target"]),
            "parseable_free": int(pred_free.isdigit()),
            "format_valid_free": int(pred_free.isdigit() and len(pred_free) == len(row["target"])),
            "pred_free_len": len(pred_free),
            "teacher_correct": teacher_correct,
            "teacher_pred_chars": teacher_pred_chars,
        })
        pred_records.append(rec)

        c1, c2, c3 = carry_vector_n_digits(int(row["a"]), int(row["b"]), 3) if row["width_in"] == 3 else (None, None, None)
        if len(row["target"]) == 4:
            pred_chars = teacher_pred_chars + [None] * (4 - len(teacher_pred_chars))
            carry_probe_rows.append({
                "experiment": name,
                "family": exp["family"],
                "extra_data_kind": exp["extra_data_kind"],
                "seed": seed,
                "suite": row["suite"],
                "a": int(row["a"]),
                "b": int(row["b"]),
                "c1": c1,
                "c2": c2,
                "c3": c3,
                "target_thousands": row["target"][0],
                "target_hundreds": row["target"][1],
                "pred_thousands": "<BOS>" if pred_chars[0] is None else pred_chars[0],
                "pred_hundreds": "<BOS>" if pred_chars[1] is None else pred_chars[1],
                "pred_tens": "<BOS>" if pred_chars[2] is None else pred_chars[2],
                "pred_ones": "<BOS>" if pred_chars[3] is None else pred_chars[3],
            })

        for i, (target_ch, pred_ch) in enumerate(zip(row["target"], teacher_pred_chars)):
            teacher_token_rows.append({
                "experiment": name,
                "family": exp["family"],
                "train_family": exp["train_family"],
                "model_kind": exp["model_kind"],
                "suite": row["suite"],
                "position_left": i,
                "target_length": len(row["target"]),
                "target_char": target_ch,
                "pred_char": "<BOS>" if pred_ch is None else pred_ch,
                "is_special_pred": int((pred_ch is None) or (pred_ch in ["+", "="])),
            })

    pred_df = pd.DataFrame(pred_records)
    pred_df.to_json(workdir / "eval_predictions.jsonl", orient="records", lines=True, force_ascii=False)


    for tpos in range(4):
        pred_df[f"target_L{tpos}"] = pred_df["target"].str[tpos].fillna("")
        pred_df[f"pred_L{tpos}"] = pred_df["pred_free"].str[tpos].fillna("")
    pred_df["thousands_correct"] = pred_df["pred_L0"] == pred_df["target_L0"]
    pred_df["hundreds_correct"] = pred_df["pred_L1"] == pred_df["target_L1"]
    pred_df["tens_correct"] = pred_df["pred_L2"] == pred_df["target_L2"]
    pred_df["ones_correct"] = pred_df["pred_L3"] == pred_df["target_L3"]
    pred_df["high2_correct"] = pred_df["thousands_correct"] & pred_df["hundreds_correct"]
    pred_df["low2_correct"] = pred_df["tens_correct"] & pred_df["ones_correct"]

    for suite, g in pred_df[pred_df["suite"].str.startswith("true3")].groupby("suite"):
        recomposition_rows.append({
            "experiment": name,
            "family": exp["family"],
            "train_family": exp["train_family"],
            "model_kind": exp["model_kind"],
            "extra_data_kind": exp["extra_data_kind"],
            "seed": seed,
            "suite": suite,
            "n": len(g),
            "exact_rate": float(g["exact_free"].mean()),
            "high2_correct_rate": float(g["high2_correct"].mean()),
            "low2_correct_rate": float(g["low2_correct"].mean()),
            "thousands_correct_rate": float(g["thousands_correct"].mean()),
            "hundreds_correct_rate": float(g["hundreds_correct"].mean()),
            "tens_correct_rate": float(g["tens_correct"].mean()),
            "ones_correct_rate": float(g["ones_correct"].mean()),
            "exact_given_high2_rate": float(g.loc[g["high2_correct"], "low2_correct"].mean()) if g["high2_correct"].any() else np.nan,
            "high2_given_low2_rate": float(g.loc[g["low2_correct"], "high2_correct"].mean()) if g["low2_correct"].any() else np.nan,
        })

    # suite summary
    for suite, g in pred_df.groupby("suite"):
        total_teacher = sum(sum(v) for v in g["teacher_correct"])
        total_teacher_n = sum(len(v) for v in g["teacher_correct"])
        row = config.copy()
        row.update({
            "suite": suite,
            "n_eval_suite": len(g),
            "runtime_sec": runtime_sec,
            "exact_free": float(g["exact_free"].mean()),
            "exact_constrained": float(g["exact_constrained"].mean()),
            "parseable_free": float(g["parseable_free"].mean()),
            "format_valid_free": float(g["format_valid_free"].mean()),
            "pred_free_len_mean": float(g["pred_free_len"].mean()),
            "teacher_token_accuracy": float(total_teacher / max(1, total_teacher_n)),
        })
        suite_rows.append(row)

        for correct_seq, target in zip(g["teacher_correct"], g["target"]):
            L = len(target)
            for i, ok in enumerate(correct_seq):
                per_position_rows.append({
                    "experiment": name,
                    "family": exp["family"],
                    "train_family": exp["train_family"],
                    "model_kind": exp["model_kind"],
                    "suite": suite,
                    "position_left": i,
                    "position_from_right": L - 1 - i,
                    "target_length": L,
                    "accuracy": int(ok),
                })

    # one-line combined summary
    total_teacher = sum(sum(v) for v in pred_df["teacher_correct"])
    total_teacher_n = sum(len(v) for v in pred_df["teacher_correct"])
    combined_rows.append({
        **config,
        "runtime_sec": runtime_sec,
        "exact_free_overall": float(pred_df["exact_free"].mean()),
        "exact_constrained_overall": float(pred_df["exact_constrained"].mean()),
        "parseable_free_overall": float(pred_df["parseable_free"].mean()),
        "format_valid_free_overall": float(pred_df["format_valid_free"].mean()),
        "pred_free_len_mean_overall": float(pred_df["pred_free_len"].mean()),
        "teacher_token_accuracy_overall": float(total_teacher / max(1, total_teacher_n)),
    })

manifest_df = pd.DataFrame(manifest_rows)
combined_df = pd.DataFrame(combined_rows)
suite_df = pd.DataFrame(suite_rows)
per_position_df = pd.DataFrame(per_position_rows)
teacher_token_df = pd.DataFrame(teacher_token_rows)

manifest_df.to_csv(output_root / "manifest.csv", index=False)
combined_df.to_csv(output_root / "combined_summary.csv", index=False)
suite_df.to_csv(output_root / "suite_summary.csv", index=False)
per_position_df.to_csv(output_root / "per_position_raw.csv", index=False)

per_position_summary = per_position_df.groupby(
    ["family", "train_family", "model_kind", "suite", "position_from_right", "target_length"]
).agg(
    mean_accuracy=("accuracy", "mean"),
    std_accuracy=("accuracy", "std"),
    n=("accuracy", "size"),
).reset_index()
per_position_summary.to_csv(output_root / "per_position_summary.csv", index=False)

teacher_token_df.to_csv(output_root / "teacher_token_raw.csv", index=False)
teacher_special_summary = teacher_token_df.groupby(
    ["family", "train_family", "model_kind", "suite", "position_left", "target_length"]
).agg(
    n=("pred_char", "size"),
    special_pred_rate=("is_special_pred", "mean"),
    bos_rate=("pred_char", lambda s: (s == "<BOS>").mean()),
    plus_rate=("pred_char", lambda s: (s == "+").mean()),
    eq_rate=("pred_char", lambda s: (s == "=").mean()),
    most_common_pred=("pred_char", lambda s: s.value_counts().idxmax()),
    most_common_rate=("pred_char", lambda s: s.value_counts().iloc[0] / len(s)),
).reset_index()
teacher_special_summary.to_csv(output_root / "teacher_special_token_summary.csv", index=False)

carry_probe_df = pd.DataFrame(carry_probe_rows)
if len(carry_probe_df):
    carry_probe_df.to_csv(output_root / "carry_probe_raw.csv", index=False)
    carry_probe_summary = carry_probe_df.groupby(
        ["family", "extra_data_kind", "suite", "seed"]
    ).agg(
        n=("a", "size"),
        pred_hundreds_eq_c2_rate=("pred_hundreds", lambda s: (s == carry_probe_df.loc[s.index, "c2"].astype(str)).mean()),
        pred_thousands_eq_c3_rate=("pred_thousands", lambda s: (s == carry_probe_df.loc[s.index, "c3"].astype(str)).mean()),
        pred_hundreds_zero_rate=("pred_hundreds", lambda s: (s == "0").mean()),
        pred_hundreds_one_rate=("pred_hundreds", lambda s: (s == "1").mean()),
        pred_thousands_zero_rate=("pred_thousands", lambda s: (s == "0").mean()),
        pred_thousands_one_rate=("pred_thousands", lambda s: (s == "1").mean()),
    ).reset_index()
    carry_probe_summary.to_csv(output_root / "carry_probe_summary.csv", index=False)

    carry_probe_mean_summary = carry_probe_summary.groupby(
        ["family", "extra_data_kind", "suite"]
    ).agg(
        seeds=("seed", "nunique"),
        pred_hundreds_eq_c2_mean=("pred_hundreds_eq_c2_rate", "mean"),
        pred_hundreds_eq_c2_std=("pred_hundreds_eq_c2_rate", "std"),
        pred_thousands_eq_c3_mean=("pred_thousands_eq_c3_rate", "mean"),
        pred_thousands_eq_c3_std=("pred_thousands_eq_c3_rate", "std"),
        pred_hundreds_zero_mean=("pred_hundreds_zero_rate", "mean"),
        pred_hundreds_one_mean=("pred_hundreds_one_rate", "mean"),
        pred_thousands_zero_mean=("pred_thousands_zero_rate", "mean"),
        pred_thousands_one_mean=("pred_thousands_one_rate", "mean"),
    ).reset_index()
    carry_probe_mean_summary.to_csv(output_root / "carry_probe_mean_summary.csv", index=False)

recomposition_df = pd.DataFrame(recomposition_rows)
if len(recomposition_df):
    recomposition_df.to_csv(output_root / "recomposition_summary.csv", index=False)
    recomposition_mean_summary = recomposition_df.groupby(
        ["family", "extra_data_kind", "suite"]
    ).agg(
        seeds=("seed", "nunique"),
        exact_rate_mean=("exact_rate", "mean"),
        exact_rate_std=("exact_rate", "std"),
        high2_correct_mean=("high2_correct_rate", "mean"),
        high2_correct_std=("high2_correct_rate", "std"),
        low2_correct_mean=("low2_correct_rate", "mean"),
        low2_correct_std=("low2_correct_rate", "std"),
        thousands_correct_mean=("thousands_correct_rate", "mean"),
        hundreds_correct_mean=("hundreds_correct_rate", "mean"),
        tens_correct_mean=("tens_correct_rate", "mean"),
        ones_correct_mean=("ones_correct_rate", "mean"),
        exact_given_high2_mean=("exact_given_high2_rate", "mean"),
        exact_given_high2_std=("exact_given_high2_rate", "std"),
    ).reset_index()
    recomposition_mean_summary.to_csv(output_root / "recomposition_mean_summary.csv", index=False)

# --- Stage11 extra analyses: subgroup composition summaries ---
cp = carry_probe_df.copy()
target_str = (cp["a"] + cp["b"]).map(lambda s: f"{int(s):04d}")
cp["target_tens"] = target_str.str[-2]
cp["target_ones"] = target_str.str[-1]
cp["pred_thousands_str"] = cp["pred_thousands"].astype(str)
cp["pred_hundreds_str"] = cp["pred_hundreds"].astype(str)
cp["pred_tens_str"] = cp["pred_tens"].astype(str)
cp["pred_ones_str"] = cp["pred_ones"].astype(str)
cp["thousands_correct"] = cp["pred_thousands_str"] == cp["target_thousands"].astype(str)
cp["hundreds_correct"] = cp["pred_hundreds_str"] == cp["target_hundreds"].astype(str)
cp["tens_correct"] = cp["pred_tens_str"] == cp["target_tens"].astype(str)
cp["ones_correct"] = cp["pred_ones_str"] == cp["target_ones"].astype(str)
cp["high2_correct"] = cp["thousands_correct"] & cp["hundreds_correct"]
cp["low2_correct"] = cp["tens_correct"] & cp["ones_correct"]
cp["exact"] = cp["high2_correct"] & cp["low2_correct"]

thousands_c2_summary = cp[cp["suite"] == "true3_thousands_carry"].groupby(
    ["family", "extra_data_kind", "seed", "c2"]
).agg(
    n=("a", "size"),
    exact_rate=("exact", "mean"),
    high2_correct_rate=("high2_correct", "mean"),
    low2_correct_rate=("low2_correct", "mean"),
    thousands_correct_rate=("thousands_correct", "mean"),
    hundreds_correct_rate=("hundreds_correct", "mean"),
    tens_correct_rate=("tens_correct", "mean"),
    ones_correct_rate=("ones_correct", "mean"),
).reset_index()
thousands_c2_summary.to_csv(output_root / "thousands_c2_summary.csv", index=False)
thousands_c2_mean_summary = thousands_c2_summary.groupby(
    ["family", "extra_data_kind", "c2"]
).agg(
    seeds=("seed", "nunique"),
    exact_rate_mean=("exact_rate", "mean"),
    exact_rate_std=("exact_rate", "std"),
    high2_correct_mean=("high2_correct_rate", "mean"),
    high2_correct_std=("high2_correct_rate", "std"),
    low2_correct_mean=("low2_correct_rate", "mean"),
    low2_correct_std=("low2_correct_rate", "std"),
    thousands_correct_mean=("thousands_correct_rate", "mean"),
    hundreds_correct_mean=("hundreds_correct_rate", "mean"),
    tens_correct_mean=("tens_correct_rate", "mean"),
    ones_correct_mean=("ones_correct_rate", "mean"),
).reset_index()
thousands_c2_mean_summary.to_csv(output_root / "thousands_c2_mean_summary.csv", index=False)

def classify_error(row):
    if row["exact"]:
        return "exact"
    parts = []
    if not row["thousands_correct"]:
        parts.append("th")
    if not row["hundreds_correct"]:
        parts.append("h")
    if not row["tens_correct"]:
        parts.append("t")
    if not row["ones_correct"]:
        parts.append("o")
    return "+".join(parts)

cp["error_pattern"] = cp.apply(classify_error, axis=1)

def try_int(x):
    try:
        return int(x)
    except Exception:
        return None

cp["pred_tens_num"] = cp["pred_tens_str"].map(try_int)
cp["target_tens_num"] = cp["target_tens"].astype(int)
cp["tens_delta"] = cp.apply(
    lambda r: (r["pred_tens_num"] - r["target_tens_num"]) if r["pred_tens_num"] is not None else np.nan,
    axis=1,
)

residual_rows = []
for (family, extra_data_kind, suite, seed), sub in cp[cp["suite"].str.startswith("true3")].groupby(
    ["family", "extra_data_kind", "suite", "seed"]
):
    mask = sub["high2_correct"] & (~sub["exact"])
    sub2 = sub[mask].copy()
    vc = sub2["error_pattern"].value_counts(normalize=True)
    tens_err = sub2[~sub2["tens_correct"]].copy()
    residual_rows.append({
        "family": family,
        "extra_data_kind": extra_data_kind,
        "suite": suite,
        "seed": seed,
        "n_high2correct_not_exact": int(len(sub2)),
        "frac_t_only": float(vc.get("t", 0.0)),
        "frac_o_only": float(vc.get("o", 0.0)),
        "frac_t_plus_o": float(vc.get("t+o", 0.0)),
        "frac_tens_abs1": float((tens_err["tens_delta"].abs() == 1).mean()) if len(tens_err) else np.nan,
    })
residual_df = pd.DataFrame(residual_rows)
residual_df.to_csv(output_root / "residual_error_pattern_summary.csv", index=False)
residual_mean_df = residual_df.groupby(
    ["family", "extra_data_kind", "suite"]
).agg(
    seeds=("seed", "nunique"),
    n_high2correct_not_exact_mean=("n_high2correct_not_exact", "mean"),
    frac_t_only_mean=("frac_t_only", "mean"),
    frac_o_only_mean=("frac_o_only", "mean"),
    frac_t_plus_o_mean=("frac_t_plus_o", "mean"),
    frac_tens_abs1_mean=("frac_tens_abs1", "mean"),
).reset_index()
residual_mean_df.to_csv(output_root / "residual_error_pattern_mean_summary.csv", index=False)

suite_mean_summary = suite_df.groupby(
    ["family", "train_family", "model_kind", "suite"]
).agg(
    seeds=("seed", "nunique"),
    exact_free_mean=("exact_free", "mean"),
    exact_free_std=("exact_free", "std"),
    exact_constrained_mean=("exact_constrained", "mean"),
    exact_constrained_std=("exact_constrained", "std"),
    parseable_free_mean=("parseable_free", "mean"),
    format_valid_free_mean=("format_valid_free", "mean"),
    pred_free_len_mean=("pred_free_len_mean", "mean"),
    teacher_token_accuracy_mean=("teacher_token_accuracy", "mean"),
    teacher_token_accuracy_std=("teacher_token_accuracy", "std"),
).reset_index()
suite_mean_summary.to_csv(output_root / "suite_mean_summary.csv", index=False)

readme = """\
Results pack contents (Stage18 large-capacity conditional-binding bridge pack):
- design_manifest.csv
- probe_design_manifest.csv
- probe_h0_preview.txt
- probe_h1_preview.txt
- probe_t1_preview.txt
- probe_t1c2_1_preview.txt
- bridge_preview.txt
- thousands_c2_summary.csv
- thousands_c2_mean_summary.csv
- residual_error_pattern_summary.csv
- residual_error_pattern_mean_summary.csv
- absolute_position_role_aliasing.csv
- absolute_position_support.csv
- coupled_position_scheme.csv
- manifest.csv
- combined_summary.csv
- suite_summary.csv
- suite_mean_summary.csv
- per_position_raw.csv
- per_position_summary.csv
- teacher_token_raw.csv
- teacher_special_token_summary.csv
- carry_probe_raw.csv
- carry_probe_summary.csv
- carry_probe_mean_summary.csv
- recomposition_summary.csv
- recomposition_mean_summary.csv
- per-experiment folders with config / vocab / train_log / predictions / checkpoint
"""
(output_root / "README_results.txt").write_text(readme, encoding="utf-8")

# --- Human-readable summary for manuscript integration ---
def infer_family_label(family_name: str) -> str:
    if "tailhigh" in family_name:
        return "tailhigh"
    if "highonly" in family_name:
        return "highonly"
    if "_tail_" in family_name or family_name.endswith("_tail_large") or family_name.endswith("_tail_small"):
        return "tail"
    if "ctrl3" in family_name:
        return "ctrl3"
    return family_name

def suite_pretty_name(suite_name: str) -> str:
    mapping = {
        "true3_hundreds_no_incarry": "true3 no-incarry",
        "true3_hundreds_with_incarry": "true3 with-incarry",
        "true3_thousands_carry": "true3 thousands-carry",
    }
    return mapping.get(suite_name, suite_name)

suite_for_story = suite_mean_summary.copy()
suite_for_story["label"] = suite_for_story["family"].map(infer_family_label)
suite_for_story = suite_for_story[suite_for_story["suite"].isin([
    "true3_hundreds_no_incarry",
    "true3_hundreds_with_incarry",
    "true3_thousands_carry",
])].copy()

recomp_for_story = recomposition_mean_summary.copy()
recomp_for_story["label"] = recomp_for_story["family"].map(infer_family_label)
recomp_for_story = recomp_for_story[recomp_for_story["suite"].isin([
    "true3_hundreds_no_incarry",
    "true3_hundreds_with_incarry",
    "true3_thousands_carry",
])].copy()

family_order = ["ctrl3", "highonly", "tail", "tailhigh"]
suite_order = [
    "true3_hundreds_no_incarry",
    "true3_hundreds_with_incarry",
    "true3_thousands_carry",
]

def paired_wins(raw_df: pd.DataFrame, suite_name: str, metric: str, left: str, right: str) -> int:
    sub = raw_df[raw_df["suite"] == suite_name].copy()
    sub["label"] = sub["family"].map(infer_family_label)
    piv = sub[sub["label"].isin([left, right])].pivot_table(
        index="seed", columns="label", values=metric, aggfunc="first"
    )
    piv = piv.dropna()
    if left not in piv.columns or right not in piv.columns:
        return 0
    return int((piv[left] > piv[right]).sum())

story_lines = []
story_lines.append("# Stage18 large-capacity bridge summary")
story_lines.append("")
story_lines.append("This run repeats the Stage18 conditional-binding ablation at the same capacity as the later tens-residual study (2-layer, width-32, 4-head).")
story_lines.append("The central question is whether the binding order ctrl3 / highonly / tail / tailhigh still holds once the model capacity is increased.")
story_lines.append("")
story_lines.append("## Exact-match means on the three true-3-digit suites")
story_lines.append("")
story_lines.append("| Suite | ctrl3 | highonly | tail | tailhigh | tailhigh - tail | tailhigh - ctrl3 | tailhigh wins vs tail (paired seeds) |")
story_lines.append("| --- | ---: | ---: | ---: | ---: | ---: | ---: | ---: |")

for suite_name in suite_order:
    sub = suite_for_story[suite_for_story["suite"] == suite_name].copy()
    exact_map = {row["label"]: float(row["exact_free_mean"]) for _, row in sub.iterrows()}
    exact_tailhigh = exact_map.get("tailhigh", np.nan)
    exact_tail = exact_map.get("tail", np.nan)
    exact_ctrl3 = exact_map.get("ctrl3", np.nan)
    wins_tail = paired_wins(suite_df, suite_name, "exact_free", "tailhigh", "tail")
    story_lines.append(
        f"| {suite_pretty_name(suite_name)} | "
        f"{exact_map.get('ctrl3', np.nan):.3f} | {exact_map.get('highonly', np.nan):.3f} | "
        f"{exact_map.get('tail', np.nan):.3f} | {exact_map.get('tailhigh', np.nan):.3f} | "
        f"{(exact_tailhigh - exact_tail):+.3f} | {(exact_tailhigh - exact_ctrl3):+.3f} | {wins_tail}/{len(BASE_SEEDS)} |"
    )

story_lines.append("")
story_lines.append("## Recomposition metric: P(exact | high2 correct)")
story_lines.append("")
story_lines.append("| Suite | ctrl3 | highonly | tail | tailhigh | tailhigh - tail | tailhigh - ctrl3 | tailhigh wins vs tail (paired seeds) |")
story_lines.append("| --- | ---: | ---: | ---: | ---: | ---: | ---: | ---: |")

for suite_name in suite_order:
    sub = recomp_for_story[recomp_for_story["suite"] == suite_name].copy()
    recomp_map = {row["label"]: float(row["exact_given_high2_mean"]) for _, row in sub.iterrows()}
    recomp_tailhigh = recomp_map.get("tailhigh", np.nan)
    recomp_tail = recomp_map.get("tail", np.nan)
    recomp_ctrl3 = recomp_map.get("ctrl3", np.nan)
    wins_tail = paired_wins(recomposition_df, suite_name, "exact_given_high2_rate", "tailhigh", "tail")
    story_lines.append(
        f"| {suite_pretty_name(suite_name)} | "
        f"{recomp_map.get('ctrl3', np.nan):.3f} | {recomp_map.get('highonly', np.nan):.3f} | "
        f"{recomp_map.get('tail', np.nan):.3f} | {recomp_map.get('tailhigh', np.nan):.3f} | "
        f"{(recomp_tailhigh - recomp_tail):+.3f} | {(recomp_tailhigh - recomp_ctrl3):+.3f} | {wins_tail}/{len(BASE_SEEDS)} |"
    )

story_lines.append("")
story_lines.append("## Plain-language interpretation")
story_lines.append("")
# determine overall ordering count
ordered_exact = 0
ordered_recomp = 0
for suite_name in suite_order:
    em = {row["label"]: float(row["exact_free_mean"]) for _, row in suite_for_story[suite_for_story["suite"] == suite_name].iterrows()}
    rm = {row["label"]: float(row["exact_given_high2_mean"]) for _, row in recomp_for_story[recomp_for_story["suite"] == suite_name].iterrows()}
    if em.get("tailhigh", -1e9) > em.get("tail", -1e9) and em.get("tail", -1e9) > max(em.get("ctrl3", -1e9), em.get("highonly", -1e9)):
        ordered_exact += 1
    if rm.get("tailhigh", -1e9) > rm.get("tail", -1e9) and rm.get("tail", -1e9) > max(rm.get("ctrl3", -1e9), rm.get("highonly", -1e9)):
        ordered_recomp += 1

story_lines.append(
    f"- Tailhigh > tail > max(ctrl3, highonly) holds on exact match in {ordered_exact}/{len(suite_order)} true-3-digit suites."
)
story_lines.append(
    f"- The same ordering holds on P(exact | high2 correct) in {ordered_recomp}/{len(suite_order)} suites."
)

if len(residual_mean_df):
    residual_story = residual_mean_df.copy()
    residual_story["label"] = residual_story["family"].map(infer_family_label)
    residual_story = residual_story[residual_story["label"] == "tailhigh"]
    if len(residual_story):
        mean_t_only = float(residual_story["frac_t_only_mean"].mean())
        mean_t_abs1 = float(residual_story["frac_tens_abs1_mean"].mean())
        story_lines.append(
            f"- Under tailhigh, the remaining high2-correct but not-exact errors are still mostly tens-only (mean fraction {mean_t_only:.3f}),"
            f" and the tens error is often one digit in magnitude (mean |Δtens|=1 fraction {mean_t_abs1:.3f})."
        )

story_lines.append("")
story_lines.append("## Recommended manuscript update if the ordering is clean")
story_lines.append("")
story_lines.append(
    "If tailhigh again dominates at this larger capacity, the paper can say that the conditional-binding story is not confined to the analytic 1-layer sandbox: the same ordering reappears at the 2-layer, width-32 capacity that supports the later tens-residual study."
)

human_summary_path = output_root / "human_readable_summary.md"
human_summary_path.write_text("\n".join(story_lines) + "\n", encoding="utf-8")

key_claim_rows = []
for suite_name in suite_order:
    em = suite_for_story[suite_for_story["suite"] == suite_name].copy()
    rm = recomp_for_story[recomp_for_story["suite"] == suite_name].copy()
    em_map = {row["label"]: float(row["exact_free_mean"]) for _, row in em.iterrows()}
    rm_map = {row["label"]: float(row["exact_given_high2_mean"]) for _, row in rm.iterrows()}
    key_claim_rows.append({
        "suite": suite_name,
        "suite_pretty": suite_pretty_name(suite_name),
        "ctrl3_exact": em_map.get("ctrl3", np.nan),
        "highonly_exact": em_map.get("highonly", np.nan),
        "tail_exact": em_map.get("tail", np.nan),
        "tailhigh_exact": em_map.get("tailhigh", np.nan),
        "tailhigh_minus_tail_exact": em_map.get("tailhigh", np.nan) - em_map.get("tail", np.nan),
        "tailhigh_minus_ctrl3_exact": em_map.get("tailhigh", np.nan) - em_map.get("ctrl3", np.nan),
        "tailhigh_vs_tail_exact_wins": paired_wins(suite_df, suite_name, "exact_free", "tailhigh", "tail"),
        "ctrl3_recomp": rm_map.get("ctrl3", np.nan),
        "highonly_recomp": rm_map.get("highonly", np.nan),
        "tail_recomp": rm_map.get("tail", np.nan),
        "tailhigh_recomp": rm_map.get("tailhigh", np.nan),
        "tailhigh_minus_tail_recomp": rm_map.get("tailhigh", np.nan) - rm_map.get("tail", np.nan),
        "tailhigh_minus_ctrl3_recomp": rm_map.get("tailhigh", np.nan) - rm_map.get("ctrl3", np.nan),
        "tailhigh_vs_tail_recomp_wins": paired_wins(recomposition_df, suite_name, "exact_given_high2_rate", "tailhigh", "tail"),
    })
pd.DataFrame(key_claim_rows).to_csv(output_root / "key_claims_largebridge.csv", index=False)

# zip
zip_path = output_root.with_suffix(".zip")
if zip_path.exists():
    zip_path.unlink()

with zipfile.ZipFile(zip_path, "w", zipfile.ZIP_DEFLATED) as zf:
    for path in output_root.rglob("*"):
        zf.write(path, path.as_posix())

print("done")
print("output_root:", output_root.resolve())
print("zip:", zip_path.resolve())
print(pd.read_csv(output_root / "suite_mean_summary.csv"))
print("human summary:", human_summary_path.resolve())

if AUTO_DOWNLOAD_ZIP:
    try:
        from google.colab import files
        files.download(str(zip_path))
    except Exception as e:
        print("auto-download skipped:", repr(e))
